In [4]:
import pandas
import numpy
import matplotlib.pyplot as plt
import sklearn
import keras
import os
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (d:\Python_Course\DL\day6-4hrs\deploy_ml\myenv\lib\site-packages\keras\preprocessing\image\__init__.py)

In [ ]:
obj = ImageDataGenerator(
    rotation_range=360,vertical_flip=True,horizontal_flip=True,zoom_range=0.3,width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3
)

base_path_bachchan = '/content/bachchan'
base_path_srk = '/content/srk'

for path in [base_path_bachchan, base_path_srk]:
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(path, filename)
            img=load_img(image_path, color_mode='rgb')
            X = img_to_array(img)
            X = numpy.expand_dims(X, axis=0)

            i=0
            for x in obj.flow(X, save_to_dir=path, save_format='jpg', save_prefix='aug_'):
                i += 1
                if i > 5:
                 break

In [ ]:
base_path_bachchan = '/content/bachchan'
base_path_srk = '/content/srk'
X = []
y = []

for path in [base_path_bachchan, base_path_srk]:
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(path, filename)
            img = Image.open(image_path)
            if img.mode == 'RGB':
                img_resized = img.resize((200,200), Image.LANCZOS)
            else:
                continue
            X.append(img_resized)
            y.append(path)


In [ ]:
image_array = []
for img in X:
    image_array.append(numpy.array(img))

X = numpy.array(image_array)
print(X.shape)
print(len(y))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2, stratify=ytrain)

In [ ]:
Xtrain = Xtrain/255.0
Xval = Xval/255.0
Xtest = Xtest/255.0

In [ ]:
Xtrain.shape

In [ ]:
Xval.shape

In [ ]:
Xtest.shape

In [ ]:
Xtrain = Xtrain.reshape(89, 200*200*3)
Xval = Xval.reshape(23, 200*200*3)
Xtest = Xtest.reshape(28, 200*200*3)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif


pipe = Pipeline(
   steps = [
        ('step3_machinelearning', RandomForestClassifier())
    ]
)

paramgrid = {

    'step3_machinelearning__bootstrap': [True, False],#done
    'step3_machinelearning__oob_score': [True, False],#done
    'step3_machinelearning__warm_start': [True, False],#done
    'step3_machinelearning__max_samples': [1.0, 0.25, 0.5, 0.75],#done
    'step3_machinelearning__n_estimators': [10, 25, 5, 50, 100], #done
    'step3_machinelearning__criterion': ['gini', 'entropy', 'log_loss'], #done
    'step3_machinelearning__max_depth': [None,2,3,4,5], #done
    'step3_machinelearning__min_samples_split': [2,5,10],#done
    'step3_machinelearning__min_samples_leaf': [1, 2, 4],#done
    'step3_machinelearning__max_features': [None, 'sqrt', 'log2'],#done
    'step3_machinelearning__max_leaf_nodes': [None, 5, 10, 20],#done
    'step3_machinelearning__min_impurity_decrease': [0.0, 0.1, 0.2],#done
    'step3_machinelearning__ccp_alpha': [0.0, 0.1, 0.2]#done
}

# Create the GridSearchCV object
random_search = RandomizedSearchCV(estimator=pipe, param_distributions=paramgrid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV to find the best hyperparameters
random_search.fit(Xtrain, ytrain)

# Print the best hyperparameters and best score
print("Best Hyperparameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

In [ ]:
import pickle

model = random_search.best_estimator_

with open('random_forest_srk_bachchan.pkl', 'wb') as f:
    pickle.dump(model, f)